In [21]:
pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.0 MB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 11.2 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 10.7 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install influxdb_client

  Using cached influxdb_client-1.48.0-py3-none-any.whl.metadata (65 kB)
  Using cached reactivex-4.0.4-py3-none-any.whl.metadata (5.5 kB)
Using cached influxdb_client-1.48.0-py3-none-any.whl (746 kB)
Using cached reactivex-4.0.4-py3-none-any.whl (217 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from datetime import timedelta
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from sklearn.preprocessing import LabelEncoder
import os

In [2]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [6]:
import os
from datetime import datetime, timedelta

import pandas as pd
from influxdb_client import InfluxDBClient
from sklearn.preprocessing import LabelEncoder

# Influx configuration
INFLUX_ORG = "wise2024"
# INFLUX_TOKEN = os.environ.get("INFLUXDB_HOST", "131.159.85.125:8086")
INFLUX_TOKEN = os.environ.get("INFLUXDB_HOST", "192.168.81.143:8086")  # home IP
INFLUX_USER = os.environ.get("INFLUXDB_USER", "admin")
INFLUX_PASS = os.environ.get("INFLUXDB_PASS", "secure_influx_iot_user")

# Sensor information
SENSOR_MAP = {
    "kitchen": {
        "battery": "1_5_10", 
        "PIR": "1_5_9"
    },
    "livingroom": {
        "PIR": "1_4_7",
        "battery": "1_4_8",
        "magnetic_switch": "1_4_11"
    },
    "bathroom": {
        "PIR": "1_3_6",
        "battery": "1_3_5"
    }
}

BUCKET_DICT = {
    "1_5_10": ["kitchen_battery"],
    "1_5_9": ["kitchen_PIR"],
    "1_4_7": ["livingroom_PIR"],
    "1_4_8": ["livingroom_battery"],
    "1_4_11": ["livingroom_magnetic_switch"],
    "1_3_6": ["bathroom_PIR"],
    "1_3_5": ["bathroom_battery"]
}

BUCKETS = ["1_5_10", "1_5_9", "1_4_7", "1_4_8", "1_4_11", "1_3_6", "1_3_5"]
PIR_BUCKETS = ["1_5_9", "1_4_7", "1_3_6"]
MAGNETIC_SWITCH_BUCKETS = ["1_4_11"]
BATTERY_BUCKETS = ["1_5_10", "1_4_8"]


def fetch_data(bucket: str,
               measurement: str,
               field: str,
               start_hours: int = 24,
               interval_hours: int = 6) -> list:
    """
    Fetch data from InfluxDB starting from `start_hours` in the past 
    and spanning `interval_hours` hours forward.

    Example:
        start_hours=24, interval_hours=6
        This will fetch data from 24 hours ago up until 18 hours ago (24 - 6).
    """
    now = datetime.utcnow()
    _start = now - timedelta(hours=start_hours)
    _stop = _start + timedelta(hours=interval_hours)

    with InfluxDBClient(
        url=INFLUX_TOKEN,
        org=INFLUX_ORG,
        username=INFLUX_USER,
        password=INFLUX_PASS,
        verify_ssl=False
    ) as client:
        query_api = client.query_api()

        params = {
            "_start": _start.isoformat(),
            "_stop": _stop.isoformat()
        }

        query = f'''
            from(bucket: "{bucket}")
            |> range(start: _start, stop: _stop)
            |> filter(fn: (r) => r["_measurement"] == "{measurement}")
            |> filter(fn: (r) => r["_type"] == "sensor-value")
            |> filter(fn: (r) => r["_field"] == "{field}")
        '''

        tables = query_api.query(query, params=params)

        results = []
        for table in tables:
            for record in table.records:
                val = {
                    "sensor": BUCKET_DICT[bucket],
                    "bucket": bucket,
                    "timestamp": record["_time"].timestamp() * 1000,
                    "value": record["_value"]
                }

                if bucket in BATTERY_BUCKETS:
                    val["field"] = record["_field"]
                    val["type"] = "battery"
                else:
                    val["type"] = "sensor"

                results.append(val)

        return results


def prepare_data_for_model(sensor_data: list) -> pd.DataFrame:
    """
    Prepare sensor data for model training.

    :param sensor_data: List of sensor-data dictionaries.
    :return: Preprocessed pandas DataFrame.
    """
    df = pd.DataFrame(sensor_data)

    # Handle cases where 'sensor' is a list by extracting the first element
    if 'sensor' in df.columns:
        df['sensor'] = df['sensor'].apply(
            lambda x: x[0] if isinstance(x, list) and len(x) > 0 else 'unknown_sensor'
        )

    # Convert timestamp from milliseconds to datetime
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Sort by timestamp
    df = df.sort_values('timestamp')

    # Encode categorical variables
    if 'sensor' in df.columns:
        le = LabelEncoder()
        try:
            df['sensor_encoded'] = le.fit_transform(df['sensor'])
        except Exception:
            df['sensor_encoded'] = 0  # Fallback if something goes wrong

    return df


def fetch_data_from_buckets(buckets: list,
                            measurement: str,
                            fields: list,
                            start_hours: int = 24,
                            interval_hours: int = 6) -> list:
    """
    Fetch data from multiple buckets for a given measurement and fields.

    :param buckets: List of bucket names.
    :param measurement: Measurement name (e.g., "PIR", "battery").
    :param fields: List of field names (e.g., ["roomID", "soc", "voltage"]).
    :param start_hours: How many hours in the past to start.
    :param interval_hours: Span in hours to fetch data.
    :return: Concatenated list of data from all buckets/fields.
    """
    all_data = []
    for bucket in buckets:
        for field in fields:
            all_data.extend(
                fetch_data(
                    bucket=bucket,
                    measurement=measurement,
                    field=field,
                    start_hours=start_hours,
                    interval_hours=interval_hours
                )
            )
    return all_data


def fetch_all_sensor_data(start_hours: int = 24,
                          interval_hours: int = 24) -> list:
    """
    Fetch all sensor data (PIR and Magnetic Switch) within the specified time range.

    :param start_hours: How many hours in the past to start fetching data.
    :param interval_hours: Time-span (in hours) to fetch data from the start point.
    :return: Aggregated list of sensor data.
    """
    all_sensor_data = []

    # Fetch PIR sensor data
    pir_data = fetch_data_from_buckets(
        buckets=PIR_BUCKETS,
        measurement="PIR",
        fields=["roomID"],  # Adjust fields as necessary
        start_hours=start_hours,
        interval_hours=interval_hours
    )
    all_sensor_data.extend(pir_data)

    # Fetch Magnetic Switch data
    magnetic_switch_data = fetch_data_from_buckets(
        buckets=MAGNETIC_SWITCH_BUCKETS,
        measurement="MagneticSwitch",
        fields=["roomID"],  # Adjust fields as necessary
        start_hours=start_hours,
        interval_hours=interval_hours
    )
    all_sensor_data.extend(magnetic_switch_data)

    return all_sensor_data



# Fetch data for the last 24 hours, with a 24-hour interval (i.e., up to now)
sensor_data = fetch_all_sensor_data(start_hours=24, interval_hours=24)
sensor_data_df = prepare_data_for_model(sensor_data)

print(sensor_data_df.head(20))


ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'Vary': 'Accept-Encoding', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': 'v2.7.11', 'X-Platform-Error-Code': 'invalid', 'Date': 'Thu, 02 Jan 2025 10:41:32 GMT', 'Transfer-Encoding': 'chunked'})
HTTP response body: b'{"code":"invalid","message":"error calling function \\"range\\" @3:16-3:49: value is not a time, got string"}'


In [5]:
# load the csv
df = pd.read_csv("data.csv", parse_dates=['timestamp'])

# parse time stamps
# df['timestamp'] = pd.to_datetime(df['timestamp'])

# Create a flag that indicates when the 'value' changes compared to the previous row
df['room_change'] = (df['value'] != df['value'].shift(1)).astype(int)

# Create a cumulative sum of the 'room_change' flag to assign a unique group ID to each consecutive block
df['group_id'] = df['room_change'].cumsum()

# Group by 'group_id' and 'value' to handle each room separately
duration_df = df.groupby(['group_id', 'value']).agg(
    start_time=('timestamp', 'min'),
    end_time=('timestamp', 'max')
).reset_index()

# Calculate duration as the difference between end_time and start_time
duration_df['duration'] = duration_df['end_time'] - duration_df['start_time']

# Convert 'duration' to total seconds for easier numerical processing
duration_df['duration_seconds'] = duration_df['duration'].apply(lambda x: pd.to_timedelta(x).total_seconds())

In [6]:
duration_df.tail(20)

,group_id,value,start_time,end_time,duration,duration_seconds
699,700,bathroom,2024-12-04 06:01:07.037,2024-12-04 06:10:10.375,0 days 00:09:03.338000,543.338
700,701,livingroomdoor,2024-12-04 06:10:54.000,2024-12-04 06:10:54.000,0 days 00:00:00,0.000
701,702,bathroom,2024-12-04 06:12:57.410,2024-12-04 06:14:26.732,0 days 00:01:29.322000,89.322
702,703,livingroombedarea,2024-12-04 07:22:04.976,2024-12-04 07:25:12.427,0 days 00:03:07.451000,187.451
703,704,livingroomdoor,2024-12-04 08:15:41.000,2024-12-04 08:15:41.000,0 days 00:00:00,0.000
704,705,livingroombedarea,2024-12-04 08:23:59.080,2024-12-04 08:24:13.803,0 days 00:00:14.723000,14.723
705,706,bathroom,2024-12-04 08:24:31.237,2024-12-04 08:33:21.808,0 days 00:08:50.571000,530.571
706,707,livingroombedarea,2024-12-04 08:33:28.532,2024-12-04 08:34:02.072,0 days 00:00:33.540000,33.540
707,708,bathroom,2024-12-04 08:34:12.971,2024-12-04 08:40:41.126,0 days 00:06:28.155000,388.155
708,709,livingroombedarea,2024-12-04 08:52:39.603,2024-12-04 09:27:19.734,0 days 00:34:40.131000,2080.131


In [44]:
# Group by 'value' (room) and calculate statistics
room_stats = duration_df.groupby('value')['duration_seconds'].agg(['mean', 'std']).reset_index()

# Handle cases where std might be NaN (e.g., only one entry for a room)
room_stats['std'] = room_stats['std'].fillna(0)

print("\nRoom Statistics:")
print(room_stats)


Room Statistics:
               value         mean          std
0           bathroom   335.878195  1632.663384
1            kitchen  1094.701454  4613.714474
2  livingroombedarea  1411.446180  5113.371932
3     livingroomdoor   166.646465   563.477420


In [46]:
# Define anomaly threshold (e.g., 3 standard deviations)
threshold = 3

# Calculate upper and lower bounds for each room
room_stats['upper_bound'] = room_stats['mean'] + threshold * room_stats['std']
room_stats['lower_bound'] = room_stats['mean'] - threshold * room_stats['std']

# Replace negative lower bounds with zero
room_stats['lower_bound'] = room_stats['lower_bound'].apply(lambda x: max(x, 0))

print("\nAdjusted Room Statistics with Bounds:")
print(room_stats)




Adjusted Room Statistics with Bounds:
               value         mean          std   upper_bound  lower_bound
0           bathroom   335.878195  1632.663384   5233.868348            0
1            kitchen  1094.701454  4613.714474  14935.844876            0
2  livingroombedarea  1411.446180  5113.371932  16751.561975            0
3     livingroomdoor   166.646465   563.477420   1857.078723            0


In [48]:
def detect_anomalies(new_data, stats_df, threshold=3):
    """
    Detect anomalies based on the duration spent in each room.
    
    Parameters:
    - new_data: DataFrame with 'value', 'start_time', 'end_time', 'duration_seconds'
    - stats_df: DataFrame with 'value', 'mean', 'std', 'upper_bound', 'lower_bound'
    - threshold: Number of standard deviations to use for anomaly detection
    
    Returns:
    - DataFrame containing anomalies
    """
    # Merge new data with statistics
    merged = new_data.merge(stats_df, on='value', how='left')
    
    # Replace NaN statistics with zero bounds (assuming no prior data)
    merged['upper_bound'] = merged['upper_bound'].fillna(0)
    merged['lower_bound'] = merged['lower_bound'].fillna(0)
    
    # Identify anomalies
    anomalies = merged[
        (merged['duration_seconds'] > merged['upper_bound']) |
        (merged['duration_seconds'] < merged['lower_bound'])
    ]
    
    return anomalies


In [49]:
# Example: Detect anomalies in the existing data
anomalies = detect_anomalies(duration_df, room_stats, threshold=3)

print("\nAnomalies Detected:")
print(anomalies)


Anomalies Detected:
     group_id              value              start_time  \
3           4            kitchen 2024-11-21 02:50:16.250   
4           5  livingroombedarea 2024-11-21 09:03:39.394   
12         13           bathroom 2024-11-22 08:28:38.224   
13         14            kitchen 2024-11-22 14:14:52.088   
74         75  livingroombedarea 2024-11-23 00:49:08.798   
129       130           bathroom 2024-11-23 17:24:45.436   
150       151  livingroombedarea 2024-11-24 00:14:56.818   
235       236           bathroom 2024-11-24 07:37:25.764   
339       340            kitchen 2024-11-25 06:23:15.014   
363       364  livingroombedarea 2024-11-26 01:47:15.327   
368       369     livingroomdoor 2024-11-26 08:27:44.000   
502       503            kitchen 2024-11-27 12:39:43.486   
515       516     livingroomdoor 2024-11-27 21:03:48.000   
540       541            kitchen 2024-11-28 12:57:08.824   
615       616            kitchen 2024-11-30 18:29:17.050   
639       640      